In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
dfs = []

for root, dirs, files in os.walk("./data"):
  for file in files:
    if "rd_hcf.csv" in file and "mimi" not in file:
      module = file.split("_")[0]
      df = pd.read_csv(os.path.join(root, file))
      df["module"] = module
      # print(df["Itr"].unique())
      dfs.append(df)
      


df = pd.concat(dfs)
display(df.columns)

# Filter out SA edge victims

In [ ]:
for module in df["module"].unique():
  module_data = df.loc[(df["module"] == module) & (df["Aggr. Type"].isin(["Upper", "Lower"]))]
  type_counts = module_data.groupby('Vic Row')['Aggr. Type'].nunique()
  incomplete_idx = type_counts[type_counts == 1].index
  incomplete_rows = module_data[module_data['Vic Row'].isin(incomplete_idx)]["Vic Row"].unique()
  break

In [ ]:
def swap_patterns(df, rows_to_swap):
  df_copy = df.copy()
  
  # Create a mask for the rows we want to modify
  mask = df_copy['Vic Row'].isin(rows_to_swap)
  
  df_copy.loc[mask & (df_copy['Data Pattern'] == '0x00000000'), 'Data Pattern'] = '_0xFFFFFFFF'
  df_copy.loc[mask & (df_copy['Data Pattern'] == '0xFFFFFFFF'), 'Data Pattern'] = '_0x00000000'

  df_copy.loc[mask & (df_copy['Data Pattern'] == '_0x00000000'), 'Data Pattern'] = '0x00000000'
  df_copy.loc[mask & (df_copy['Data Pattern'] == '_0xFFFFFFFF'), 'Data Pattern'] = '0xFFFFFFFF'

  return df_copy

In [ ]:
module_mapping = {
  "axmicr02"  : "Mfr. M 8Gb E-Die",
  "hisasa00"  : "Mfr. S 8Gb B-Die",        
  "sasa05"    : "Mfr. S 8Gb D-Die",      
  "hisasa01"  : "Mfr. S 8Gb E-Die",      
  "hisasa02"  : "Mfr. S 16Gb M-Die",          
  "hisasa03"  : "Mfr. S 16Gb A-Die",        
  "sasa23"    : "Mfr. S 16Gb B-Die",
  "sasa29"    : "Mfr. S 16Gb C-Die",      
  "hyhy0c"    : "Mfr. H 8Gb C-Die",        
  "hyhy13"    : "Mfr. H 8Gb D-Die",      
  "hyhy1e"    : "Mfr. H 16Gb A-Die",    
  "hyhy03"    : "Mfr. H 16Gb C-Die",
}

In [ ]:
label_map = {
    '0x00000000': '0 →1 ',
    '0xFFFFFFFF': '1 → 0',
}


In [ ]:
default_palette = ['#2ecc71',    # green
                  '#3498db',    # blue
]

label_color_map = {
    "S" : "#e74c3c",
    "H" : "#9b59b6",
    "M" : "#e67e22",
}

## List all modules

In [ ]:
data = df.loc[df["Aggr. Type"] == "Double"]
modules = df['module'].unique()

## Figure 3

In [ ]:
# Set maximum display rows
pd.set_option('display.max_rows', 100)  # Change number as needed

# Set maximum display columns
pd.set_option('display.max_columns', 20)  # Change number as needed

# Set width of the display in characters
pd.set_option('display.width', 1000)

# Set maximum column width
pd.set_option('display.max_colwidth', 100)

In [ ]:
from matplotlib.ticker import FuncFormatter
import numpy as np
def format_k(x, p):
  return f'{int(x/1000)}K' if x >= 1000 else str(int(x))

In [ ]:
data = df.loc[df["Aggr. Type"] == "Double"]
modules = df['module'].unique()

# Calculate grid dimensions
n_cols = 4  # You can adjust this
n_rows = (len(modules) + n_cols - 1) // n_cols

# Create figure and subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 2*n_rows))
# _axes = axes
axes = axes.flatten()  # Flatten the 2D array of axes
# plt.subplots_adjust(hspace=0.7)

avg_dfs = []

# Create boxplot for each group
for idx, module in enumerate(list(module_mapping.keys())):
    # Filter edge victims
    ss_data = df[(df['module'] == module) & (df["Aggr. Type"].isin(["Upper", "Lower"]))]
    type_counts = ss_data.groupby('Vic Row')['Aggr. Type'].nunique()
    incomplete_idx = type_counts[type_counts == 1].index
    incomplete_rows = ss_data[ss_data['Vic Row'].isin(incomplete_idx)]["Vic Row"].unique()

    group_data = df.loc[(df['module'] == module) & (df["Aggr. Type"] == "Double")]
    group_data = group_data[~(group_data['Vic Row'].isin(incomplete_rows))]

    # Handle Micron true- anti-
    if module == "axmicr02":
      ret_df = pd.read_csv("./data/axmicr02_retention.csv")
      ret_df = ret_df.loc[ret_df["tWAIT"] == 4096]

      anti_df = ret_df.loc[(ret_df["Pattern"] == "00000000") & (ret_df["NumBitflips"] >= 10)]
      true_df = ret_df.loc[(ret_df["Pattern"] == "FFFFFFFF") & (ret_df["NumBitflips"] >= 10)]

      anti_rows = anti_df["Row"].unique() + 1024
      true_rows = true_df["Row"].unique() + 1024

      group_data = swap_patterns(group_data, list(anti_rows))

    adf = group_data.groupby(['Data Pattern'])['HC'].mean().reset_index()
    adf["module"] = module
    avg_dfs.append(adf)

    # Create boxplot on the corresponding subplot
    sns.boxplot(data=group_data, x="Data Pattern", y="HC" , ax=axes[idx], order=['0x00000000', '0xFFFFFFFF'], palette=default_palette)

    mfr = module_mapping[module].split()[1]
    if idx not in [0, 1, 8]:
      axes[idx].set_title(f'{" ".join(module_mapping[module].split()[2:])}', weight='bold', color=label_color_map[mfr])
    else:
      axes[idx].set_title(f'{module_mapping[module]}', weight='bold', color=label_color_map[mfr])
    axes[idx].yaxis.set_major_formatter(FuncFormatter(format_k))
    axes[idx].set_xlabel('')
    axes[idx].set_ylabel('')
    if idx % 4 == 0:
      axes[idx].set_ylabel(r'$\text{HC}_{\text{First}}$')

    y_min, y_max = axes[idx].get_ylim()
    margin = (y_max - y_min) * 0.05  # 5% margin
    axes[idx].set_ylim(y_min - margin, y_max + margin)
    axes[idx].set_yticks(np.linspace(y_min, y_max, 5, endpoint=True))


    # Make borders thicker
    for spine in axes[idx].spines.values():
        spine.set_linewidth(1.6)  # Adjust this number to control thickness
        spine.set_color('black')
    # Get current x-tick labels
    current_labels = [label.get_text() for label in axes[idx].get_xticklabels()]
    
    # Replace labels using the mapping
    new_labels = [label_map.get(label, label) for label in current_labels]
    
    # Set new labels
    axes[idx].set_xticklabels(new_labels, weight='bold')

# Remove empty subplots if any
for idx in range(len(modules), len(axes)):
    fig.delaxes(axes[idx])

for ax in axes:
  ax.grid(True, axis='y', linestyle='--', linewidth=1.0)
  ax.tick_params(axis='y', width=1.5, direction='in', color='black')

# # fig.supxlabel('Bitfip Direction')
# import matplotlib.patches as mpatches
# patch1 = mpatches.Patch(color="#e74c3c",  label='Mfr. S')
# patch2 = mpatches.Patch(color="#9b59b6",  label='Mfr. H')
# patch3 = mpatches.Patch(color="#e67e22",  label='Mfr. M')

# # Add a legend at the top center of the figure spanning across columns (ncol=3)
# fig.legend(handles=[patch1, patch2, patch3],
#            loc='upper center', 
#            ncol=3,
#            bbox_to_anchor=(0.5, 1.05))  # Adjust the y-value to position above the figure

# legend = ax.legend(handlelength=0, handletextpad=1,
#                    loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1.05))

# for text, color in zip(legend.get_texts(), ["#e74c3c", "#9b59b6", "#e67e22"]):
#   text.set_color(color)
# # plt.show()

plt.tight_layout(h_pad=1.3, w_pad=1.2)
import matplotlib.lines as mlines
# Retrieve each subplot's position (in figure coordinates)
positions = []
for i, ax in enumerate(axes):
  if i % 4 == 0:
     positions.append(ax.get_position())
# Loop through consecutive rows to compute and draw the dashed line between them
for i in range(len(positions) - 1):
  # Compute the y position: average between the bottom of the upper subplot
  # and the top of the lower subplot
  y = (positions[i].y0 + positions[i+1].y1) / 2
  # Create a horizontal dashed line across the figure
  x = [0, 1] if i != 0 else [0, 0.271]
  line = mlines.Line2D(x, [y, y],
                        color='black', linestyle='--', linewidth=1,
                        transform=fig.transFigure)
  fig.add_artist(line)

pos = axes[0].get_position()
print(pos)
print(axes[1].get_position())
line = mlines.Line2D([0.271, 0.271], [1, (positions[0].y0 + positions[1].y1) / 2],
                     color='black', linestyle='--', linewidth=1,
                     transform=fig.transFigure)
fig.add_artist(line)

plt.show()
fig.savefig("DS_HCF.png", pad_inches=0.05)
fig.savefig("DS_HCF.pdf", pad_inches=0.05)

## Results for Table 2

In [ ]:
avg_df = pd.concat(avg_dfs)
avg_df.columns

# Split into two dataframes
df_00 = avg_df[avg_df['Data Pattern'] == '0x00000000'][['module', 'HC']].rename(columns={'HC': 'HC_00'})
df_FF = avg_df[avg_df['Data Pattern'] == '0xFFFFFFFF'][['module', 'HC']].rename(columns={'HC': 'HC_FF'})

# Merge and calculate normalized values
result = df_FF.merge(df_00, on='module')
result['normalized_HC'] = result['HC_00'] / result['HC_FF']
result['diff'] = 1 - result['normalized_HC']
result['diff'] = result['diff'].apply(lambda x: f"{x:.1%}")

geometric_mean = np.exp(np.log(result['normalized_HC']).mean())
print(geometric_mean)

## Figure 4

In [ ]:
dfs = []

for root, dirs, files in os.walk("./data"):
  for file in files:
    if "rd_ber.csv" in file and "mimi" not in file:
      module = file.split("_")[0]
      df = pd.read_csv(os.path.join(root, file))
      df["module"] = module
      dfs.append(df)

df = pd.concat(dfs)

In [ ]:
df.columns

In [ ]:
dfs = []

for root, dirs, files in os.walk("./data"):
  for file in files:
    if "rd_hcf.csv" in file and "mimi" not in file:
      module = file.split("_")[0]
      _df = pd.read_csv(os.path.join(root, file))
      _df["module"] = module
      dfs.append(_df)

hcf_df = pd.concat(dfs)

In [ ]:
data = df.loc[df["Aggr. Type"] == "Double"]
modules = df['module'].unique()

# Calculate grid dimensions
n_cols = 4  # You can adjust this
n_rows = (len(modules) + n_cols - 1) // n_cols

# Create figure and subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 2*n_rows))
axes = axes.flatten()  # Flatten the 2D array of axes


avg_dfs = []

# Create boxplot for each group
for idx, module in enumerate(list(module_mapping.keys())):
    # Filter edge victims
    ss_data = hcf_df[(hcf_df['module'] == module) & (hcf_df["Aggr. Type"].isin(["Upper", "Lower"]))]
    type_counts = ss_data.groupby('Vic Row')['Aggr. Type'].nunique()
    incomplete_idx = type_counts[type_counts == 1].index
    incomplete_rows = ss_data[ss_data['Vic Row'].isin(incomplete_idx)]["Vic Row"].unique()

    group_data = df.loc[(df['module'] == module) & (df["Aggr. Type"] == "Double")]
    group_data = group_data[~(group_data['Vic Row'].isin(incomplete_rows))]



    # Handle Micron true- anti-
    if module == "axmicr02":
      ret_df = pd.read_csv("./data/axmicr02_retention.csv")
      ret_df = ret_df.loc[ret_df["tWAIT"] == 4096]

      anti_df = ret_df.loc[(ret_df["Pattern"] == "00000000") & (ret_df["NumBitflips"] >= 10)]
      true_df = ret_df.loc[(ret_df["Pattern"] == "FFFFFFFF") & (ret_df["NumBitflips"] >= 10)]

      anti_rows = anti_df["Row"].unique() + 1024
      true_rows = true_df["Row"].unique() + 1024
      group_data = swap_patterns(group_data, list(anti_rows))

    adf = group_data.groupby(['Data Pattern'])['Num. Bitflips'].mean().reset_index()
    adf["module"] = module
    avg_dfs.append(adf)


    # Create boxplot on the corresponding subplot
    sns.boxplot(data=group_data, x="Data Pattern", y="Num. Bitflips" , ax=axes[idx], order=['0x00000000', '0xFFFFFFFF'], palette=default_palette)
    mfr = module_mapping[module].split()[1]
    if idx not in [0, 1, 8]:
      axes[idx].set_title(f'{" ".join(module_mapping[module].split()[2:])}', weight='bold', color=label_color_map[mfr])
    else:
      axes[idx].set_title(f'{module_mapping[module]}', weight='bold', color=label_color_map[mfr])
    axes[idx].yaxis.set_major_formatter(FuncFormatter(format_k))
    axes[idx].set_xlabel('')
    axes[idx].set_ylabel('')
    if idx % 4 == 0:
       axes[idx].set_ylabel(r'Number of Bitflips')

    y_min, y_max = axes[idx].get_ylim()
    margin = (y_max - y_min) * 0.05  # 5% margin
    axes[idx].set_ylim(max(y_min - margin, 0), y_max + margin)
    axes[idx].set_yticks(np.linspace(y_min, y_max, 5, endpoint=True))


    # Make borders thicker
    for spine in axes[idx].spines.values():
        spine.set_linewidth(1.6)  # Adjust this number to control thickness
        spine.set_color('black')
    # Get current x-tick labels
    current_labels = [label.get_text() for label in axes[idx].get_xticklabels()]
    
    # Replace labels using the mapping
    new_labels = [label_map.get(label, label) for label in current_labels]
    
    # Set new labels
    axes[idx].set_xticklabels(new_labels, weight="bold")

# Remove empty subplots if any
for idx in range(len(modules), len(axes)):
    fig.delaxes(axes[idx])

for ax in axes:
  ax.grid(True, axis='y', linestyle='--', linewidth=1.0)
  ax.tick_params(axis='y', width=1, direction='in', color='black')

plt.tight_layout(h_pad=1.3, w_pad=1.2)
import matplotlib.lines as mlines
# Retrieve each subplot's position (in figure coordinates)
positions = []
for i, ax in enumerate(axes):
  if i % 4 == 0:
     positions.append(ax.get_position())
# Loop through consecutive rows to compute and draw the dashed line between them
for i in range(len(positions) - 1):
  # Compute the y position: average between the bottom of the upper subplot
  # and the top of the lower subplot
  y = (positions[i].y0 + positions[i+1].y1) / 2
  # Create a horizontal dashed line across the figure
  x = [0, 1] if i != 0 else [0, 0.271]
  line = mlines.Line2D(x, [y, y],
                        color='black', linestyle='--', linewidth=1,
                        transform=fig.transFigure)
  fig.add_artist(line)

pos = axes[0].get_position()
print(pos)
print(axes[1].get_position())
line = mlines.Line2D([0.271, 0.271], [1, (positions[0].y0 + positions[1].y1) / 2],
                     color='black', linestyle='--', linewidth=1,
                     transform=fig.transFigure)
fig.add_artist(line)
plt.show()
fig.savefig("DS_BER.png", pad_inches=0.05)
fig.savefig("DS_BER.pdf", pad_inches=0.05)

## Table 3

In [ ]:
avg_df = pd.concat(avg_dfs)
avg_df.columns

# Split into two dataframes
df_00 = avg_df[avg_df['Data Pattern'] == '0x00000000'][['module', 'Num. Bitflips']].rename(columns={'Num. Bitflips': 'BER_00'})
df_FF = avg_df[avg_df['Data Pattern'] == '0xFFFFFFFF'][['module', 'Num. Bitflips']].rename(columns={'Num. Bitflips': 'BER_FF'})

# Merge and calculate normalized values
result = df_FF.merge(df_00, on='module')
result['normalized_BER'] = result['BER_FF'] / result['BER_00']

geometric_mean = np.exp(np.log(result['normalized_BER']).mean())
print(geometric_mean)

## Figure 5

In [ ]:
dfs = []

for root, dirs, files in os.walk("./data"):
  for file in files:
    if "rd_rp.csv" in file and "mimi" not in file:
      module = file.split("_")[0]
      df = pd.read_csv(os.path.join(root, file))
      df["module"] = module
      dfs.append(df)

df = pd.concat(dfs)

In [ ]:
data = df.loc[df["Aggr. Type"].isin(["Upper", "Lower"])]
modules = data['module'].unique()

# Calculate grid dimensions
n_cols = 4  # You can adjust this
n_rows = (len(modules) + n_cols - 1) // n_cols

# Create figure and subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 2*n_rows))
axes = axes.flatten()  # Flatten the 2D array of axes


avg_dfs = []

# Create boxplot for each group
for idx, module in enumerate(list(module_mapping.keys())):
    # Filter edge victims
    ss_data = hcf_df[(hcf_df['module'] == module) & (hcf_df["Aggr. Type"].isin(["Upper", "Lower"]))]
    type_counts = ss_data.groupby('Vic Row')['Aggr. Type'].nunique()
    incomplete_idx = type_counts[type_counts == 1].index
    incomplete_rows = ss_data[ss_data['Vic Row'].isin(incomplete_idx)]["Vic Row"].unique()

    group_data = df.loc[(df['module'] == module) & (df["Aggr. Type"].isin(["Upper", "Lower"]))]
    group_data = group_data[~(group_data['Vic Row'].isin(incomplete_rows))]



    # Handle Micron true- anti-
    if module == "axmicr02":
      ret_df = pd.read_csv("./data/axmicr02_retention.csv")
      ret_df = ret_df.loc[ret_df["tWAIT"] == 4096]

      anti_df = ret_df.loc[(ret_df["Pattern"] == "00000000") & (ret_df["NumBitflips"] >= 10)]
      true_df = ret_df.loc[(ret_df["Pattern"] == "FFFFFFFF") & (ret_df["NumBitflips"] >= 10)]

      anti_rows = anti_df["Row"].unique() + 1024
      true_rows = true_df["Row"].unique() + 1024

      group_data = swap_patterns(group_data, list(anti_rows))

    adf = group_data.groupby(['Data Pattern', "Aggr. Type"])['Num. Bitflips'].mean().reset_index()
    adf["module"] = module
    avg_dfs.append(adf)


    # Create boxplot on the corresponding subplot
    group_data = group_data[(group_data['module'] == module)]
    if idx == 0:
        sns_plot = sns.boxplot(data=group_data, x="Data Pattern", y="Num. Bitflips", 
                            hue="Aggr. Type", ax=axes[idx], 
                            order=['0x00000000', '0xFFFFFFFF'], 
                            palette=['#2185D0', '#DB2828'])
        # Get the legend handles and labels from the first plot
        handles, labels = axes[0].get_legend_handles_labels()
        # Remove the legend from the first plot
        axes[0].get_legend().remove()
    else:
        sns_plot = sns.boxplot(data=group_data, x="Data Pattern", y="Num. Bitflips", 
                            hue="Aggr. Type", ax=axes[idx], 
                            order=['0x00000000', '0xFFFFFFFF'], 
                            palette=['#2185D0', '#DB2828'],
                            legend=False)
        

    mfr = module_mapping[module].split()[1]
    if idx not in [0, 1, 8]:
      axes[idx].set_title(f'{" ".join(module_mapping[module].split()[2:])}', weight='bold', color=label_color_map[mfr])
    else:
      axes[idx].set_title(f'{module_mapping[module]}', weight='bold', color=label_color_map[mfr])
    # axes[idx].yaxis.set_major_formatter(FuncFormatter(format_k))
    axes[idx].set_xlabel('')
    axes[idx].set_ylabel('')
    if idx % 4 == 0:
       axes[idx].set_ylabel('Num. Bitflips')

    y_min, y_max = axes[idx].get_ylim()
    margin = (y_max - y_min) * 0.05  # 5% margin
    axes[idx].set_ylim(y_min - margin, y_max + margin)
    axes[idx].set_yticks(np.linspace(max(y_min - margin, 0), y_max, 5, endpoint=True))


    # Make borders thicker
    for spine in axes[idx].spines.values():
        spine.set_linewidth(1.6)  # Adjust this number to control thickness
        spine.set_color('black')
    # Get current x-tick labels
    current_labels = [label.get_text() for label in axes[idx].get_xticklabels()]
    
    # Replace labels using the mapping
    new_labels = [label_map.get(label, label) for label in current_labels]
    
    # Set new labels
    axes[idx].set_xticklabels(new_labels, weight="bold")

    from matplotlib.ticker import MaxNLocator
    axes[idx].yaxis.set_major_locator(MaxNLocator(nbins=6, integer=True))

# Remove empty subplots if any
for idx in range(len(modules), len(axes)):
    fig.delaxes(axes[idx])

for ax in axes:
  ax.grid(True, axis='y', linestyle='--', linewidth=1.0)
  ax.tick_params(axis='y', width=1, direction='in', color='black')

legend = fig.legend(handles, [label + " Aggressor" for label in labels],  # Add 'Type: ' prefix to labels
                   bbox_to_anchor=(0.53, 1.00),  # Position legend at top
                   loc='lower center',          # Center the legend
                   ncol=len(labels),            # Make all items in one row
                   frameon=True,                # Add frame
                   edgecolor='black',           # Frame color
                   prop={'weight': 'bold', 'size': 14})
legend.get_frame().set_linewidth(2)

plt.tight_layout(h_pad=1.3, w_pad=1.2)
import matplotlib.lines as mlines
# Retrieve each subplot's position (in figure coordinates)
positions = []
for i, ax in enumerate(axes):
  if i % 4 == 0:
     positions.append(ax.get_position())
# Loop through consecutive rows to compute and draw the dashed line between them
for i in range(len(positions) - 1):
  # Compute the y position: average between the bottom of the upper subplot
  # and the top of the lower subplot
  y = (positions[i].y0 + positions[i+1].y1) / 2
  # Create a horizontal dashed line across the figure
  x = [0, 1] if i != 0 else [0, 0.273]
  line = mlines.Line2D(x, [y, y],
                        color='black', linestyle='--', linewidth=1,
                        transform=fig.transFigure)
  fig.add_artist(line)

pos = axes[0].get_position()
print(pos)
print(axes[1].get_position())
line = mlines.Line2D([0.273, 0.273], [1, (positions[0].y0 + positions[1].y1) / 2],
                     color='black', linestyle='--', linewidth=1,
                     transform=fig.transFigure)
fig.add_artist(line)
plt.show()
fig.savefig("DS_RP2.png", bbox_inches='tight', pad_inches=0.05)
fig.savefig("DS_RP2.pdf", bbox_inches='tight', pad_inches=0.05)

In [ ]:
avg_df = pd.concat(avg_dfs)
avg_df.columns

# Split into two dataframes
df_00 = avg_df[(avg_df['Data Pattern'] == '0xFFFFFFFF') & (avg_df['Aggr. Type'] == "Upper")][['module', 'Num. Bitflips']].rename(columns={'Num. Bitflips': 'BER_Upper'})
df_FF = avg_df[(avg_df['Data Pattern'] == '0xFFFFFFFF') & (avg_df['Aggr. Type'] == "Lower")][['module', 'Num. Bitflips']].rename(columns={'Num. Bitflips': 'BER_Lower'})

# Merge and calculate normalized values
result = df_FF.merge(df_00, on='module')
result['normalized_BER'] = result['BER_Upper'] / result['BER_Lower']

geometric_mean = np.exp(np.log(result['normalized_BER']).mean())
print(geometric_mean)

## Table 4

In [ ]:
dfs = []

for root, dirs, files in os.walk("./data"):
  for file in files:
    if "ds_ber_sweep.csv" in file and "mimi" not in file and "nxsasa" not in file:
      module = file.split("_")[0]
      df = pd.read_csv(os.path.join(root, file))
      df["module"] = module
      df = df.drop(["Temp", "Itr"], axis=1)
      dfs.append(df)

sweep_df = pd.concat(dfs)

In [ ]:
dfs = []

for root, dirs, files in os.walk("./data"):
  for file in files:
    if "rd_hcf.csv" in file and "mimi" not in file and "nxsasa" not in file:
      module = file.split("_")[0]
      df = pd.read_csv(os.path.join(root, file))
      df["module"] = module
      dfs.append(df)

hcf_df = pd.concat(dfs)
# hcf_df = hcf_df.loc[hcf_df["Aggr. Type"] == "Double"]
# hcf_df = hcf_df.groupby(["Vic Row", "module"])["HC"].min().reset_index()

hcf_df.columns

In [ ]:
processed_hcf = []

for idx, module in enumerate(list(module_mapping.keys())):
    ss_data = hcf_df[(hcf_df['module'] == module) & (hcf_df["Aggr. Type"].isin(["Upper", "Lower"]))]
    type_counts = ss_data.groupby('Vic Row')['Aggr. Type'].nunique()
    incomplete_idx = type_counts[type_counts == 1].index
    incomplete_rows = ss_data[ss_data['Vic Row'].isin(incomplete_idx)]["Vic Row"].unique()

    group_data = hcf_df.loc[(hcf_df['module'] == module)]
    group_data = group_data[~(group_data['Vic Row'].isin(incomplete_rows))]

    # Handle Micron true- anti-
    if module == "axmicr02":
      ret_df = pd.read_csv("./data/axmicr02_retention.csv")
      ret_df = ret_df.loc[ret_df["tWAIT"] == 4096]

      anti_df = ret_df.loc[(ret_df["Pattern"] == "00000000") & (ret_df["NumBitflips"] >= 10)]
      true_df = ret_df.loc[(ret_df["Pattern"] == "FFFFFFFF") & (ret_df["NumBitflips"] >= 10)]

      anti_rows = anti_df["Row"].unique() + 1024
      true_rows = true_df["Row"].unique() + 1024

      group_data = swap_patterns(group_data, list(anti_rows))

    group_data = group_data.loc[(group_data["Aggr. Type"] == "Double") & (group_data["Data Pattern"] == "0x00000000")]
    group_data = group_data.groupby(["Vic Row", "module"])["HC"].min().reset_index()
    processed_hcf.append(group_data)

processed_hcf_df = pd.concat(processed_hcf)
sdf = pd.merge(processed_hcf_df, sweep_df, on=["Vic Row", "module"], how="left", suffixes=('_hcf', '_tp'))

In [ ]:
out_sdf = sdf.groupby(["module"])[["HC_hcf", "HC_tp"]].mean().reset_index()
out_sdf["Normalized"] = out_sdf["HC_tp"] / out_sdf["HC_hcf"] 
geometric_mean = np.exp(np.log(out_sdf['Normalized']).mean())
print(geometric_mean)

In [ ]:
out_sdf['module'] = pd.Categorical(out_sdf['module'], categories=list(module_mapping.keys()), ordered=True)
out_sdf = out_sdf.sort_values('module')